# Test LLVM AST Notebook

## Author: Yiannis Charalambous


In [1]:
import os
from clang.cindex import Config
import clang.native
import clang.cindex
import sys
from typing import NamedTuple


In [2]:
# Connect the Python API of Clang to the libclang.so file bundled in the libclang PyPI package.
Config.library_file = os.path.join(
    os.path.dirname(clang.native.__file__),
    "libclang.so",
)

module_path = os.path.abspath(os.path.join(".."))
if module_path not in sys.path:
    sys.path.append(module_path)


In [3]:
FILE = "../samples/threading.c"


def get_declarations_local(root: clang.cindex.Cursor) -> list[clang.cindex.Cursor]:
    declarations: list[clang.cindex.Cursor] = []
    declarations_raw: set[str] = {}
    # Scan all direct symbols in root.
    for child in root.get_children():
        # print(f"Scanning: {child.spelling}")
        node: clang.cindex.Cursor = child
        kind: clang.cindex.CursorKind = node.kind
        # Check if it is actually from the file.
        if (
            kind.is_declaration()
            and node.storage_class == clang.cindex.StorageClass.NONE
        ):
            print(
                f"Found {node.spelling} [line={node.location.line}, col={node.location.column}]"
            )
            loc: clang.cindex.SourceRange = node.extent
            end: clang.cindex.SourceLocation = loc.end
            start: clang.cindex.SourceLocation = loc.start
            print(
                f"Start: {start.offset}, End: {end.offset}, Range: {end.offset - start.offset}"
            )
            print()
            declarations.append(node)
    return declarations


index: clang.cindex.Index = clang.cindex.Index.create()
tu: clang.cindex.TranslationUnit = index.parse(FILE)
root: clang.cindex.Cursor = tu.cursor
declarations: clang.cindex.Cursor = get_declarations_local(root)

print(f"Total {len(declarations)}")


Found a [line=4, col=5]
Start: 42, End: 47, Range: 5

Found b [line=4, col=8]
Start: 42, End: 50, Range: 8

Found __VERIFIER_atomic_acquire [line=5, col=6]
Start: 52, End: 134, Range: 82

Found c [line=10, col=7]
Start: 135, End: 224, Range: 89

Found d [line=17, col=11]
Start: 225, End: 236, Range: 11

Found main [line=18, col=5]
Start: 238, End: 363, Range: 125

Total 6


## Reversing Reach AST To Source Code

The only issue I have found, multiple declarations in one statement need to be recognized and the nodes combined:

```c
int a, b;
```


In [4]:
with open(FILE) as file:
    source_code: str = file.read()


def get_node_source_code(source_code: str, node: clang.cindex.Cursor) -> str:
    loc: clang.cindex.SourceRange = node.extent
    start: clang.cindex.SourceLocation = loc.start
    end: clang.cindex.SourceLocation = loc.end
    return source_code[start.offset : end.offset]


for node in declarations:
    print(f"Code for {node.displayname}:")
    print("```")
    print(get_node_source_code(source_code, node))
    print("```")
    print()


Code for a:
```
int a
```

Code for b:
```
int a, b
```

Code for __VERIFIER_atomic_acquire():
```
void __VERIFIER_atomic_acquire(void)
{
    __VERIFIER_assume(a == 0);
    a = 1;
}
```

Code for c(void *):
```
void *c(void *arg)
{
    ;
    __VERIFIER_atomic_acquire();
    b = 1;
    return NULL;
}
```

Code for d:
```
pthread_t d
```

Code for main():
```
int main()
{
    pthread_create(&d, 0, c, 0);
    __VERIFIER_atomic_acquire();
    if (!b)
        assert(0);
    return 0;
}
```



# Test Code


In [5]:
from esbmc_ai_lib.frontend.ast import ClangAST


In [12]:
file = "../samples/threading.c"
cast = ClangAST(file)
functions = cast.get_fn_decl()

for fn in functions:
    print(str(fn) + "\n")
    # Seems like different cursors have the same translation unit...
    print(fn.cursor)
    print(fn.cursor.translation_unit)


__VERIFIER_atomic_acquire()

c(arg: void *)

main()



# Test Code 2


In [7]:
file = "./samples/typedefs.c"
cast = ClangAST(file)
functions = cast.get_all_decl()

for fn in functions:
    print(fn)


main()
typedef (_Float32) float {}
typedef (_Float64) double {}
typedef (_Float32x) double {}
typedef (_Float64x) long double {}
typedef () div_t {}
typedef (div_t) div_t {}
typedef () ldiv_t {}
typedef (ldiv_t) ldiv_t {}
typedef () lldiv_t {}
typedef (lldiv_t) lldiv_t {}
typedef (__u_char) unsigned char {}
typedef (__u_short) unsigned short {}
typedef (__u_int) unsigned int {}
typedef (__u_long) unsigned long {}
typedef (__int8_t) signed char {}
typedef (__uint8_t) unsigned char {}
typedef (__int16_t) short {}
typedef (__uint16_t) unsigned short {}
typedef (__int32_t) int {}
typedef (__uint32_t) unsigned int {}
typedef (__int64_t) long {}
typedef (__uint64_t) unsigned long {}
typedef (__int_least8_t) signed char {}
typedef (__uint_least8_t) unsigned char {}
typedef (__int_least16_t) short {}
typedef (__uint_least16_t) unsigned short {}
typedef (__int_least32_t) int {}
typedef (__uint_least32_t) unsigned int {}
typedef (__int_least64_t) long {}
typedef (__uint_least64_t) unsigned long 

In [8]:
file = "./samples/typedefs.c"
cast: ClangAST = ClangAST(file)
includes = cast.get_include_directives()

for include in includes:
    print(include)


#include "/usr/include/stdlib.h"
#include "/usr/include/assert.h"
